In [1]:
import os
import sys
import glob

try:
    sys.path.append(glob.glob('../carla/PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass


import carla

import math
import time
import hydra
import logging
import argparse


from numpy import random
from omegaconf import DictConfig, OmegaConf, ValidationError

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from core.pgm import PGM
from utils.weather import Weather
from core.vehicle_manager import VehicleManager
from core.pedestrian_manager import PedestrianManager
from utils.common import get_actor_blueprints

from hydra.core.config_store import ConfigStore
from schemas.pgm_schema import PGMModel
from schemas.weather_schema import WeatherSchema, SunSchema

c:\Users\aadim\.conda\envs\carla-pgm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def main(cfg: DictConfig):
    logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)
    
    client = carla.Client(cfg.host, cfg.port)
    client.set_timeout(10.0)
    
    pgm = PGM(cfg.model)
    pgm_variables = pgm.get_variables()
    pgm_states = pgm.get_states()
    traffic_state = pgm.predict_state(
        [pgm_variables.Traffic, pgm_variables.Speed], 
        evidence={
            pgm_variables.Rain: pgm_states.Rain.NO_RAIN.name
        }
    )

    if traffic_state[pgm_variables.Traffic] == pgm_states.Traffic.LOW.name:
        num_of_vehicles = 10
    else:
        num_of_vehicles = 30

    if traffic_state[pgm_variables.Speed] == pgm_states.Speed.LOW.name:
        speed = -50
    else:
        speed = 10

    vcfg = hydra.compose(config_name="vehicle_manager.yaml")
    # print(vcfg)
    tm = VehicleManager(client=client, cfg=vcfg)

    pcfg = hydra.compose(config_name="pedestrian_manager.yaml")
    # print(pcfg)
    pm = PedestrianManager(client=client, cfg=pcfg)
    try:
        world = client.get_world()

        tm.spawn_vehicles(number_of_vehicles=num_of_vehicles)
        pm.spawn_walkers(number_of_walkers=pcfg.number_of_walkers)

        # Dynamic Weather
        speed_factor = 1.0
        update_freq = 0.5 / speed_factor

        weather = Weather(world.get_weather())
        weather.setWeather(cfg.weather)

        elapsed_time = 0.0

        tm.update_global_speed(speed)

        synchronous_master = world.get_settings().synchronous_mode
        while True:
            if not cfg.asynch and synchronous_master:
                world.tick()
            else:
                timestamp = world.wait_for_tick(seconds=30.0).timestamp
                elapsed_time += timestamp.delta_seconds
                if elapsed_time > update_freq:
                    # weather.tick(speed_factor * elapsed_time)

                    # world.set_weather(weather.weather)
                    # sys.stdout.write('\r' + str(weather) + 12 * ' ')
                    # sys.stdout.flush()
                    elapsed_time = 0.0

    finally:

        tm.destroy()
        pm.destroy()
        time.sleep(0.5)

In [13]:
try:
    cs = ConfigStore.instance()
    cs.store(group="weather", name="base_weather_model", node=WeatherSchema)
    cs.store(group="weather/sun", name="base_sun_model", node=SunSchema)
    cs.store(group="model", name="base_pgm_model", node=PGMModel)
    with hydra.initialize(version_base=None, config_path="../conf"):
        cfg = hydra.compose(
            config_name="config", 
            overrides=[
                "weather=clear",
                "weather/sun=day"
                
                ])
        print(cfg)
        main(cfg)
except KeyboardInterrupt:
    pass
finally:
    print('\ndone.')

{'weather': {'cloudiness': 10.0, 'rain': 0.0, 'puddles': 0.0, 'wind': 5.0, 'fog': 0.0, 'wetness': 0.0, 'sun': {'azimuth': 0.0, 'altitude': 45.0}}, 'model': {'edges': [['R', 'T'], ['R', 'S']], 'states': {'R': ['NO_RAIN', 'LIGHT_RAIN', 'HEAVY_RAIN'], 'T': ['LOW', 'HEAVY'], 'S': ['LOW', 'HIGH']}, 'cpd': {'R': {'name': 'Rain', 'cardinality': 3, 'probabilities': [[0.2], [0.5], [0.3]], 'evidence': None, 'evidence_card': None}, 'T': {'name': 'Traffic', 'cardinality': 2, 'probabilities': [[0.1, 0.4, 0.9], [0.9, 0.6, 0.1]], 'evidence': ['R'], 'evidence_card': None}, 'S': {'name': 'Speed', 'cardinality': 2, 'probabilities': [[0.1, 0.6, 0.8], [0.9, 0.4, 0.2]], 'evidence': ['R'], 'evidence_card': None}}}, 'host': '127.0.0.1', 'port': 2000, 'asynch': False, 'dynamic_weather': False}


Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
ERROR: Spawn failed because of collision at spawn position



Added 30 vehicles
Spawned 9 walkers.

Destroying 30 vehicles
Destroying 9 walkers

done.
